<a href="https://colab.research.google.com/github/pyrated03/IISc-Incremental-Learning/blob/main/CAVR_via_DRML_Naive_Incremental.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Import necessary packages
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
import os

from torchvision import datasets, transforms
import torchvision.transforms as T
import torch
import torch.nn as nn
import torchvision
from torchvision.utils import make_grid
from torchvision.utils import save_image    
from IPython.display import Image
import random


import torch.optim as optim
import torch.nn.functional as F
from torchvision import *
import time
import copy
from torch.optim import lr_scheduler
import pickle
from torchvision.io import read_image


# MNIST DATASET IMPORTING

In [ ]:
# Define a transform to normalize the data
transform = transforms.Compose([transforms.Resize((32,32)),
                                transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])

# Download and load the training data
trainset_mnist = datasets.MNIST("/content/drive/MyDrive/IISc Internship/MNIST DATA/train_set/", download=True, train=True, transform=transform)
testset_mnist = datasets.MNIST("/content/drive/MyDrive/IISc Internship/MNIST DATA/test_set", download=True, train=False, transform=transform)

indices = list(range(len(trainset_mnist)))
# spl = int(0.9*len(trainset_mnist))
random.shuffle(indices)
train_sampler = indices[:10000]
val_sampler = indices[10001:11000]

trainloader_mnist = torch.utils.data.DataLoader(trainset_mnist, sampler=train_sampler, batch_size=64)
valloader_mnist = torch.utils.data.DataLoader(trainset_mnist, sampler=val_sampler, batch_size=64)
testloader_mnist = torch.utils.data.DataLoader(testset_mnist, batch_size=64, shuffle=True)

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
mnist_data = {"train":trainloader_mnist, "val":valloader_mnist, "test":testloader_mnist}
# mnist_data = {"train":trainloader_mnist, "test":testloader_mnist}
mnist_sizes = {"train":64*len(trainloader_mnist), "val":64*len(valloader_mnist), "test":64*len(testloader_mnist)}

In [ ]:
mnist_data['train']

## MNIST-M DATASET IMPORTING

In [ ]:
class MNIST_M(torch.utils.data.Dataset):
    def __init__(self, root, train, transform=None):
        self.train = train
        self.transform = transform
        if train:
            self.image_dir = os.path.join(root, 'mnist_m_train')
            self.labels_file = os.path.join(root, "mnist_m_train_labels.txt")
        else:
            self.image_dir = os.path.join(root, 'mnist_m_test')
            self.labels_file = os.path.join(root, "mnist_m_test_labels.txt")

        with open(self.labels_file, "r") as fp:
        	content = fp.readlines()
        self.mapping = list(map(lambda x: (x[0], int(x[1])), [c.strip().split() for c in content]))

    def __len__(self):
        return len(self.mapping)

    def __getitem__(self, idx):
        image, labels = self.mapping[idx]
        image = os.path.join(self.image_dir, image)
        image = self.transform(read_image(image))
        return image, labels

    def _load_data(self):
        data = read_image_file(self.image_dir)
        targets = read_label_file(self.labels_file)

        return data, targets

In [ ]:
root_dir = "/content/drive/MyDrive/IISc Internship/MNIST-M DATA/mnist_m/"
trainset_mnistm = MNIST_M(root=root_dir, train=True,
                            transform=transforms.Compose([
                            transforms.Resize((32,32)),
                            transforms.ToPILImage(),
                            transforms.ToTensor(),
                            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                            ]))
testset_mnistm = MNIST_M(root=root_dir, train=False,
                            transform=transforms.Compose([
                            transforms.Resize((32,32)),
                            transforms.ToPILImage(),
                            transforms.ToTensor(),
                            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                            ]))
from torch.utils.data.sampler import SubsetRandomSampler

# dataset_size = len(trainset_mnistm)
dataset_size = 10000
train_indices = list(range(dataset_size))
validation_split = 0.1
split = int(np.floor(validation_split * dataset_size))
val_indices = list(range(dataset_size, (dataset_size + split)))

shuffle_dataset = True
random_seed= 42
batch_size = 64
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(train_indices)
    np.random.shuffle(val_indices)

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)
trainloader_mnistm = torch.utils.data.DataLoader(trainset_mnistm, batch_size=batch_size, sampler=train_sampler)#, drop_last=True)
valloader_mnistm = torch.utils.data.DataLoader(trainset_mnistm, batch_size=batch_size, sampler=valid_sampler)#, drop_last=True)

testloader_mnistm = torch.utils.data.DataLoader(testset_mnistm, batch_size=batch_size, shuffle=True)

In [ ]:
mnistm_data = {"train":trainloader_mnistm, "val":valloader_mnistm, "test":testloader_mnistm}
# mnist_data = {"train":trainloader_mnist, "test":testloader_mnist}
mnistm_sizes = {"train":64*len(trainloader_mnistm), "val":64*len(valloader_mnistm), "test":64*len(testloader_mnistm)}

## SYN DATASET IMPORTING

In [ ]:
# !tar -xzvf "/content/drive/MyDrive/IISc Internship/SYN DATA/c-digits.tar.gz" -C "/content/drive/MyDrive/IISc Internship/SYN DATA/"     #[run this cell to extract tar.gz files]
# !unzip "/content/drive/MyDrive/IISc Internship/SYN DATA/c-digits.zip" -d "/content/drive/MyDrive/IISc Internship/SYN DATA/"

Streaming output truncated to the last 5000 lines.
c-digits/mnistm/train/1/00017802.png
c-digits/mnistm/train/1/00024647.png
c-digits/mnistm/train/1/00019176.png
c-digits/mnistm/train/1/00012885.png
c-digits/mnistm/train/1/00044724.png
c-digits/mnistm/train/1/00028609.png
c-digits/mnistm/train/1/00011954.png
c-digits/mnistm/train/1/00057069.png
c-digits/mnistm/train/1/00022502.png
c-digits/mnistm/train/1/00047790.png
c-digits/mnistm/train/1/00018647.png
c-digits/mnistm/train/1/00034282.png
c-digits/mnistm/train/1/00025003.png
c-digits/mnistm/train/1/00048600.png
c-digits/mnistm/train/1/00047551.png
c-digits/mnistm/train/1/00041454.png
c-digits/mnistm/train/1/00041894.png
c-digits/mnistm/train/1/00033918.png
c-digits/mnistm/train/1/00054353.png
c-digits/mnistm/train/1/00047446.png
c-digits/mnistm/train/1/00006311.png
c-digits/mnistm/train/1/00043905.png
c-digits/mnistm/train/1/00057203.png
c-digits/mnistm/train/1/00007089.png
c-digits/mnistm/train/1/00007188.png
c-digits/mnistm/train/1/

## SVHN DATASET IMPORTING

In [ ]:
transform  = transforms.Compose([transforms.Resize((32,32)),
                                transforms.ToTensor(),
                                # transforms.Grayscale(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])
trainset_svhn = datasets.SVHN("/content/drive/MyDrive/IISc Internship/SVHN DATA/train_set/", download=True, split='train',transform=transform)
testset_svhn = datasets.SVHN("/content/drive/MyDrive/IISc Internship/SVHN DATA/test_set/", download=True, split='test',transform=transform)
extra_svhn = datasets.SVHN("/content/drive/MyDrive/IISc Internship/SVHN DATA/extra/", download=True, split='extra',transform=transform)

indices = list(range(len(trainset_svhn)))
# spl = int(0.9*len(trainset_svhn))
random.shuffle(indices)
train_sampler = indices[:10000]
val_sampler = indices[10001:11000]

trainloader_svhn = torch.utils.data.DataLoader(trainset_svhn, sampler = train_sampler, batch_size=64)
valloader_svhn = torch.utils.data.DataLoader(trainset_svhn, sampler = val_sampler, batch_size=64)

testloader_svhn = torch.utils.data.DataLoader(testset_svhn, batch_size=64, shuffle=True)
extraloader_svhn = torch.utils.data.DataLoader(extra_svhn, batch_size=64, shuffle=True)


Using downloaded and verified file: /content/drive/MyDrive/IISc Internship/SVHN DATA/train_set/train_32x32.mat
Using downloaded and verified file: /content/drive/MyDrive/IISc Internship/SVHN DATA/test_set/test_32x32.mat
Using downloaded and verified file: /content/drive/MyDrive/IISc Internship/SVHN DATA/extra/extra_32x32.mat


In [ ]:
svhn_data = {"train":trainloader_svhn, "val":valloader_svhn, "test":testloader_svhn, "extra":extraloader_svhn}
svhn_sizes = {"train":64*len(trainloader_svhn), "val":64*len(valloader_svhn), "test":64*len(testloader_svhn), "extra":64*len(extraloader_svhn)}

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch.cuda.is_available()

True

## Defining ResNet Model

In [ ]:
!pip3 install resnet_pytorch

In [ ]:
model = models.resnet18(pretrained=False)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### **DEFINING THE FINAL LAYER FOR CLASSIFICATION**

In [ ]:
model.fc = nn.Sequential(nn.Linear(512,10)) #, nn.Softmax(7))
# model
model = model.to(device)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
data = mnist_data
dataset_sizes = mnist_sizes

In [ ]:
def train_model_mnist(model, criterion, softmax, optimizer, scheduler, device, num_epochs=5):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in data[phase]:
                inputs = np.repeat(inputs[...], 3, -3)
                inputs = inputs.to(device)
                labels = labels.to(device)
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    # outputs = softmax(outputs)
                    _, preds = torch.max(outputs, 1)
                    # outputs = softmax(outputs)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                # print("pred: ",preds) 
                # print("label: ", labels.data)
                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data).item()
                # running_corrects += torch.sum(preds == labels)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / (dataset_sizes[phase])
            epoch_acc = running_corrects*1.0 / (dataset_sizes[phase])

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
criteria = nn.CrossEntropyLoss()
softmax = nn.Softmax()
# Observe that all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)
# optimizer = optim.Adam(model.parameters(), lr=0.0001)

scheduler = lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)
# Number of epochs
eps=10


In [ ]:
model = train_model_mnist(model,criteria,softmax, optimizer, scheduler, device, num_epochs=eps) 

Epoch 0/9
----------


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


train Loss: 1.3911 Acc: 0.5962
val Loss: 0.6900 Acc: 0.8301

Epoch 1/9
----------
train Loss: 0.5117 Acc: 0.8868
val Loss: 0.3876 Acc: 0.8896

Epoch 2/9
----------
train Loss: 0.3065 Acc: 0.9284
val Loss: 0.2820 Acc: 0.9053

Epoch 3/9
----------
train Loss: 0.2183 Acc: 0.9494
val Loss: 0.2327 Acc: 0.9160

Epoch 4/9
----------
train Loss: 0.1666 Acc: 0.9612
val Loss: 0.2044 Acc: 0.9248

Epoch 5/9
----------
train Loss: 0.1323 Acc: 0.9706
val Loss: 0.1853 Acc: 0.9297

Epoch 6/9
----------
train Loss: 0.1075 Acc: 0.9777
val Loss: 0.1724 Acc: 0.9287

Epoch 7/9
----------
train Loss: 0.0890 Acc: 0.9826
val Loss: 0.1622 Acc: 0.9316

Epoch 8/9
----------
train Loss: 0.0747 Acc: 0.9858
val Loss: 0.1550 Acc: 0.9326

Epoch 9/9
----------
train Loss: 0.0635 Acc: 0.9888
val Loss: 0.1491 Acc: 0.9316

Training complete in 2m 21s
Best val Acc: 0.932617


In [ ]:
def train_model(model, criterion, softmax, optimizer, scheduler, device, num_epochs=5):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in data[phase]:
                # inputs = np.repeat(inputs[...], 3, -3)
                inputs = inputs.to(device)
                labels = labels.to(device)
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    # outputs = softmax(outputs)
                    _, preds = torch.max(outputs, 1)
                    # outputs = softmax(outputs)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                # print("pred: ",preds) 
                # print("label: ", labels.data)
                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data).item()
                # running_corrects += torch.sum(preds == labels)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / (dataset_sizes[phase])
            epoch_acc = running_corrects*1.0 / (dataset_sizes[phase])

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
data = mnistm_data
dataset_sizes = mnistm_sizes

In [ ]:
model = train_model(model,criteria,softmax, optimizer, scheduler, device, num_epochs=eps)

Epoch 0/9
----------
train Loss: 1.6408 Acc: 0.4421
val Loss: 1.3423 Acc: 0.5234

Epoch 1/9
----------
train Loss: 1.2089 Acc: 0.5866
val Loss: 1.1260 Acc: 0.6006

Epoch 2/9
----------
train Loss: 0.9736 Acc: 0.6777
val Loss: 0.9948 Acc: 0.6494

Epoch 3/9
----------
train Loss: 0.7998 Acc: 0.7417
val Loss: 0.8808 Acc: 0.6895

Epoch 4/9
----------
train Loss: 0.6679 Acc: 0.7893
val Loss: 0.8312 Acc: 0.6992

Epoch 5/9
----------
train Loss: 0.5672 Acc: 0.8221
val Loss: 0.7627 Acc: 0.7256

Epoch 6/9
----------
train Loss: 0.4771 Acc: 0.8555
val Loss: 0.7172 Acc: 0.7324

Epoch 7/9
----------
train Loss: 0.4155 Acc: 0.8794
val Loss: 0.6910 Acc: 0.7490

Epoch 8/9
----------
train Loss: 0.3614 Acc: 0.8953
val Loss: 0.6701 Acc: 0.7568

Epoch 9/9
----------
train Loss: 0.3118 Acc: 0.9147
val Loss: 0.6504 Acc: 0.7637

Training complete in 54m 11s
Best val Acc: 0.763672


In [ ]:
data = svhn_data
dataset_sizes = svhn_sizes

In [ ]:
model = train_model(model,criteria,softmax, optimizer, scheduler, device, num_epochs=eps) 

Epoch 0/9
----------
train Loss: 2.0595 Acc: 0.3360
val Loss: 1.6994 Acc: 0.4131

Epoch 1/9
----------
train Loss: 1.4109 Acc: 0.5163
val Loss: 1.3822 Acc: 0.5107

Epoch 2/9
----------
train Loss: 1.1055 Acc: 0.6291
val Loss: 1.2651 Acc: 0.5508

Epoch 3/9
----------
train Loss: 0.8991 Acc: 0.7065
val Loss: 1.2020 Acc: 0.5752

Epoch 4/9
----------
train Loss: 0.7412 Acc: 0.7675
val Loss: 1.1480 Acc: 0.6045

Epoch 5/9
----------
train Loss: 0.6315 Acc: 0.8118
val Loss: 1.1144 Acc: 0.6025

Epoch 6/9
----------
train Loss: 0.5588 Acc: 0.8390
val Loss: 1.0900 Acc: 0.6221

Epoch 7/9
----------
train Loss: 0.4913 Acc: 0.8642
val Loss: 1.0749 Acc: 0.6240

Epoch 8/9
----------
train Loss: 0.4376 Acc: 0.8846
val Loss: 1.0744 Acc: 0.6289

Epoch 9/9
----------
train Loss: 0.3948 Acc: 0.8952
val Loss: 1.0678 Acc: 0.6445

Training complete in 2m 23s
Best val Acc: 0.644531


In [ ]:
running_corrects_test = 0
phase = "test"
for input_test_final, label_test_final in mnist_data[phase]:
    input_test_final = np.repeat(input_test_final[...], 3, -3)
    input_test_final = input_test_final.float().to(device)
    label_test_final = label_test_final.to(device)
    # print("validation:",iter_valid)
    outputs_test_final = model(input_test_final)
    _, preds_test_final = torch.max(outputs_test_final, 1)
    running_corrects_test += torch.sum(preds_test_final == label_test_final.data)

epoch_acc_test = running_corrects_test*1.0 / (mnist_sizes[phase])
print(epoch_acc_test)

tensor(0.6593, device='cuda:0')


In [ ]:
running_corrects_test = 0
phase = "test"
for input_test_final, label_test_final in mnistm_data[phase]:
    # inputs = np.repeat(inputs[...], 3, -3)
    input_test_final = input_test_final.float().to(device)
    label_test_final = label_test_final.to(device)
    # print("validation:",iter_valid)
    outputs_test_final = model(input_test_final)
    _, preds_test_final = torch.max(outputs_test_final, 1)
    running_corrects_test += torch.sum(preds_test_final == label_test_final.data)

epoch_acc_test = running_corrects_test*1.0 / (mnistm_sizes[phase])
print(epoch_acc_test)

tensor(0.5186, device='cuda:0')


In [ ]:
running_corrects_test = 0
phase = "test"
for input_test_final, label_test_final in svhn_data[phase]:
    # inputs = np.repeat(inputs[...], 3, -3)
    input_test_final = input_test_final.float().to(device)
    label_test_final = label_test_final.to(device)
    # print("validation:",iter_valid)
    outputs_test_final = model(input_test_final)
    _, preds_test_final = torch.max(outputs_test_final, 1)
    running_corrects_test += torch.sum(preds_test_final == label_test_final.data)

epoch_acc_test = running_corrects_test*1.0 / (svhn_sizes[phase])
print(epoch_acc_test)

tensor(0.6638, device='cuda:0')
